In [11]:
import json
import cv2
import os
import requests
from urllib.parse import urlparse
from urllib.parse import urljoin
import os
import shutil
import numpy as np


In [12]:
def load_labelme_json(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

In [13]:
def download_image(image_url, json_file_path, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Extract the image filename from the URL
    image_filename = os.path.basename(image_url)

    # Construct the full path to the image based on the JSON file's path
    json_directory = os.path.dirname(json_file_path)
    image_path = os.path.join(json_directory, os.pardir, image_filename)
    full_image_path = os.path.abspath(image_path)

    # Ensure that the image file exists and is not empty
    if os.path.exists(full_image_path) and os.path.getsize(full_image_path) > 0:
        # Check if the image already exists in the output directory
        destination_path = os.path.join(output_directory, image_filename)
        if not os.path.exists(destination_path):
            shutil.copy(full_image_path, destination_path)
        return destination_path
    else:
        print(f"Image file '{full_image_path}' is missing or empty.")
        return None

In [14]:
def extract_and_save_labeled_regions(json_data, image, cut_output_directory):
    shapes = json_data['shapes']

    os.makedirs(cut_output_directory, exist_ok=True)

    image_name = os.path.splitext(os.path.basename(json_data['imagePath']))[0]

    # Create a counter for the cut region image filenames
    cut_filename_counter = 0

    for shape in shapes:
        label = shape['label']
        points = shape['points']

        print(f"Label: {label}")
        print(f"Points: {points}")

        # Convert the points to a NumPy array
        points = np.array(points, dtype=np.int32)

        if len(points) > 0:
            x, y, w, h = cv2.boundingRect(points)
            region = image[y:y+h, x:x+w]

            # Generate a unique filename for the cut region image
            cut_filename = os.path.join(cut_output_directory, f'{image_name}_{cut_filename_counter + 1}.jpg')

            print(f"Cut filename: {cut_filename}")
            try:   
                cv2.imwrite(cut_filename, region)
            except Exception as e:
                print(f"An error occurred while saving the cut region: {e}")


          
            
            # Save the cut region as a separate image in the cut output directory
            

            # Increment the cut region image filename counter
            cut_filename_counter += 1

In [16]:
if __name__ == '__main__':
    json_directory = r'C:\Users\hamza\Downloads\test part 2\files'
    labeled_output_directory = r'C:\Users\hamza\Downloads\full imeges\used images'
    cut_output_directory = r'C:\Users\hamza\Downloads\full imeges\cutted images'

    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            json_file_path = os.path.join(json_directory, filename)
            json_data = load_labelme_json(json_file_path)
            image_url = json_data['imagePath']
            image_path = download_image(image_url, json_file_path, labeled_output_directory)

            if image_path:
                image = cv2.imread(image_path)
                extract_and_save_labeled_regions(json_data, image, cut_output_directory)

            print(f"Processed: {filename}")

    print("Processing completed for all JSON files.")

TypeError: list indices must be integers or slices, not str